<a href="https://colab.research.google.com/github/yuntianhe2A/thesis/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = open("/content/drive/MyDrive/FLLM/train_sentences/all_sentences.txt", 'r')
file_data = file.readlines()
file.close()
len(file_data)

1573766

In [3]:
!pip install transformers
import torch
import random
from transformers import GPT2LMHeadModel, GPT2TokenizerFast,GPT2Tokenizer
from sklearn.model_selection import train_test_split
import numpy as np
from pprint import pprint
from tqdm import tqdm
import math
import zlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.3 MB/s eta 0:00:00


In [ ]:
#@title 
# device = torch.device("cuda") 
# model1 = GPT2LMHeadModel.from_pretrained("gpt2-xl",return_dict=True).to(device)
# model2 = GPT2LMHeadModel.from_pretrained("gpt2",return_dict=True).to(device)
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", bos_token="<|startoftext|>", eos_token="<|endoftext|>",
#                                               pad_token="<|pad|>")
# model1.resize_token_embeddings(len(tokenizer))
# model2.resize_token_embeddings(len(tokenizer))

# MODEL_PATH="/content/drive/MyDrive/server_2512.pt"
# model2.load_state_dict(
#     torch.load(MODEL_PATH, map_location=device))

# model1.eval()
# model2.eval()



In [ ]:
generate_file_name="/content/drive/MyDrive/FLLM/generation/50_tem_0.5_2000.txt"
file1 = open(generate_file_name, 'r')
phone_num_generate_sentences = file1.readlines()

In [5]:
device = torch.device("cuda") 

In [4]:
def main(N,batch_size):
    
    print(f"using device: {device}")

    
    # number of tokens to generate
    seq_len = 56

    # sample from the top_k tokens output by the model
    top_k = 40

    print("Loading GPT2...")
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.padding_side = "left" 
    tokenizer.pad_token = tokenizer.eos_token

    # model1 = GPT2LMHeadModel.from_pretrained('gpt2-xl', return_dict=True).to(device)
    # model1.config.pad_token_id = model1.config.eos_token_id
    model1 = GPT2LMHeadModel.from_pretrained('gpt2', return_dict=True).to(device)
    # model1.eval()
    model1.eval()
    
    samples = []
    scores = {"XL": [], "S": [], "Lower": [], "zlib": []}

    num_batches = int(np.ceil(N / batch_size))
    with tqdm(total=N) as pbar:
        for i in range(num_batches):
            # encode the prompts
            prompts = ["<|endoftext|>"] * batch_size
            input_len = 1
            inputs = tokenizer(prompts, return_tensors="pt", padding=True)

            # batch generation
            output_sequences = model1.generate(
                input_ids=inputs['input_ids'].to(device),
                attention_mask=inputs['attention_mask'].to(device),
                max_length=input_len + seq_len,
                do_sample=True, 
                top_k=top_k, 
                top_p=1.0
            )

            texts = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

            for text in texts:
                # perplexity of GPT2-XL and GPT2-S
                p1 = calculatePerplexity(text, model1, tokenizer)
                # p2 = calculatePerplexity(text, model2, tokenizer)

                # perplexity on lower-case sample
                p_lower = calculatePerplexity(text.lower(), model1, tokenizer)

                # Zlib "entropy" of sample
                zlib_entropy = len(zlib.compress(bytes(text, 'utf-8')))

                samples.append(text)
                scores["XL"].append(p1)
                # scores["S"].append(p2)
                scores["Lower"].append(p_lower)
                scores["zlib"].append(zlib_entropy)

            pbar.update(batch_size)

    scores["XL"] = np.asarray(scores["XL"])
    # scores["S"] = np.asarray(scores["S"].cpu())
    scores["Lower"] = np.asarray(scores["Lower"])
    scores["zlib"] = np.asarray(scores["zlib"])

    # Sort by perplexity
    metric = -np.log(scores["XL"])
    print(f"======== top sample by XL perplexity: ========")
    print_best(metric, samples, "PPL", scores["XL"])
    print()
    print()

    # # Sort by ratio of log perplexities of S and XL models
    # metric = np.log(scores["S"]) / np.log(scores["XL"])
    # print(f"======== top sample by ratio of S and XL perplexities: ========")
    # print_best(metric, samples, "PPL-XL", scores["XL"], "PPL-S", scores["S"])
    # print()
    # print()

    # Sort by ratio of log perplexities of lower-case and normal-case perplexities 
    metric = np.log(scores["Lower"]) / np.log(scores["XL"])
    print(f"======== top sample by ratio of lower-case and normal-case perplexities: ========")
    print_best(metric, samples, "PPL-XL", scores["XL"], "PPL-XL-Lower", scores["Lower"])
    print()
    print()

    # Sort by ratio of Zlib entropy and XL perplexity
    metric = scores["zlib"] / np.log(scores["XL"])
    print(f"======== top sample by ratio of Zlib entropy and XL perplexity: ========")
    print_best(metric, samples, "PPL-XL", scores["XL"], "Zlib", scores["zlib"])

In [10]:
main(200000,500)

using device: cuda
Loading GPT2...


  0%|          | 0/200000 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  0%|          | 500/200000 [00:18<2:03:29, 26.93it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  0%|          | 1000/200000 [00:37<2:02:58, 26.97it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  1%|          | 1500/200000 [00:55<2:03:16, 26.84it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end

======== top sample by XL perplexity: ========
1: PPL=nan, , score=nan

'.'


2: PPL=nan, , score=nan

'.'


3: PPL=nan, , score=nan

'.'


4: PPL=nan, , score=nan

'.'


5: PPL=nan, , score=nan

'...'


6: PPL=nan, , score=nan

'.'


7: PPL=nan, , score=nan

'.'


8: PPL=nan, , score=nan

'.'


9: PPL=nan, , score=nan

'...'


10: PPL=nan, , score=nan

'..................'


11: PPL=nan, , score=nan

'.'


12: PPL=nan, , score=nan

'.'


13: PPL=nan, , score=nan

'.'


14: PPL=nan, , score=nan

'.'


15: PPL=nan, , score=nan

'.'


16: PPL=nan, , score=nan

'.'


17: PPL=nan, , score=nan

'.'


18: PPL=nan, , score=nan

'.'


19: PPL=nan, , score=nan

'.'


20: PPL=nan, , score=nan

'.'


21: PPL=nan, , score=nan

'.'


22: PPL=nan, , score=nan

'.'


23: PPL=nan, , score=nan

'..................'


24: PPL=nan, , score=nan

'.'


25: PPL=nan, , score=nan

'.'


26: PPL=nan, , score=nan

'.'


27: PPL=nan, , score=nan

'..'


28: PPL=nan, , score=nan

'.'


29: PPL=nan, , score=nan

'

In [ ]:
# def main(all_texts):

#     samples = []
#     scores = {"XL": [], "S": [], "Lower": [], "zlib": []}
#     i=0
#     for text in all_texts:
#         i+=1
#         if i%100==0:
#             print(i)
#         # perplexity of GPT2-XL and GPT2-S
#         p1 = calculatePerplexity(text, model1)
#         p2 = calculatePerplexity(text, model2)

#         # perplexity on lower-case sample
#         p_lower = calculatePerplexity(text.lower(), model1)

#         # Zlib "entropy" of sample
#         zlib_entropy = len(zlib.compress(bytes(text, 'utf-8')))

#         samples.append(text)
#         scores["XL"].append(p1)
#         scores["S"].append(p2)
#         scores["Lower"].append(p_lower)
#         scores["zlib"].append(zlib_entropy)
#     scores["XL"] = np.asarray(scores["XL"])
#     scores["S"] = np.asarray(scores["S"])
#     scores["Lower"] = np.asarray(scores["Lower"])
#     scores["zlib"] = np.asarray(scores["zlib"])

#     # Sort by perplexity
#     metric = -np.log(scores["XL"])
#     print(f"======== top sample by XL perplexity: ========")
#     print_best(metric, samples, "PPL", scores["XL"])
#     print()
#     print()

#     # Sort by ratio of log perplexities of S and XL models
#     metric = np.log(scores["S"]) / np.log(scores["XL"])
#     print(f"======== top sample by ratio of S and XL perplexities: ========")
#     print_best(metric, samples, "PPL-XL", scores["XL"], "PPL-S", scores["S"])
#     print()
#     print()

#     # Sort by ratio of log perplexities of lower-case and normal-case perplexities 
#     metric = np.log(scores["Lower"]) / np.log(scores["XL"])
#     print(f"======== top sample by ratio of lower-case and normal-case perplexities: ========")
#     print_best(metric, samples, "PPL-XL", scores["XL"], "PPL-XL-Lower", scores["Lower"])
#     print()
#     print()

#     # Sort by ratio of Zlib entropy and XL perplexity
#     metric = scores["zlib"] / np.log(scores["XL"])
#     print(f"======== top sample by ratio of Zlib entropy and XL perplexity: ========")
#     print_best(metric, samples, "PPL-XL", scores["XL"], "Zlib", scores["zlib"])

In [1]:

def calculatePerplexity(sentence, model,tokenizer):
    """
    exp(loss)
    """
    input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)
    input_ids = input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    return torch.exp(loss).item()


In [6]:
def print_best(metric, samples, name1, scores1, name2=None, scores2=None, n=50):
    """
    print the `n` best samples according to the given `metric`
    """
    idxs = np.argsort(metric)[::-1][:n]

    for i, idx in enumerate(idxs):
        if scores2 is not None:
            print(f"{i+1}: {name1}={scores1[idx]:.3f}, {name2}={scores2[idx]:.3f}, score={metric[idx]:.3f}")
        else:
            print(f"{i+1}: {name1}={scores1[idx]:.3f}, , score={metric[idx]:.3f}")

        print()
        #for line in samples[idx].split("\n"):
        #    print(f"\t {line.rstrip()}")
        pprint(samples[idx])
        print()
        print()

In [ ]:
file = open("/content/drive/MyDrive/FLLM/traing_sentences/number_sentences.txt", 'r')
number_sentences=file.readlines()
file.close()